In [ ]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy
from concurrent.futures import ThreadPoolExecutor

def create_particles(num_particles, width, height):
    """ランダムな位置、サイズ、速度を持つパーティクルを生成する。

    Args:
        num_particles: 生成するパーティクルの数。
        width: フレームの幅。
        height: フレームの高さ。

    Returns:
        パーティクルの情報を格納したNumPy配列。
        配列の各行は、パーティクルのx座標、y座標、サイズ、速度を表す。
    """
    particles = np.random.randint(0, width, (num_particles, 2))
    sizes = np.random.randint(2, 5, num_particles)
    speeds = np.random.randint(1, 5, num_particles)
    return np.column_stack((particles, sizes, speeds))

def update_particles(particles, width, height, energy):
    """音楽のエネルギーに基づいてパーティクルの位置を更新する。

    Args:
        particles: パーティクルの情報を格納したNumPy配列。
        width: フレームの幅。
        height: フレームの高さ。
        energy: 音楽のエネルギーレベル。

    Returns:
        更新されたパーティクルの情報を格納したNumPy配列。
    """
    particles[:, 1] -= particles[:, 3] * energy
    mask = particles[:, 1] < 0
    particles[mask, 1] = height
    particles[mask, 0] = np.random.randint(0, width, mask.sum())
    return particles

def draw_particles(frame, particles, color):
    """フレーム上にパーティクルを描画する。

    Args:
        frame: 描画対象のフレーム。
        particles: パーティクルの情報を格納したNumPy配列。
        color: パーティクルの色。
    """
    for p in particles:
        cv2.circle(frame, (p[0], p[1]), p[2], color, -1)

def create_mosaic_mask(height, width, num_blocks=30, min_block_size=20, max_block_size=150):
    """モザイク効果に使用するマスクを生成する。

    Args:
        height: マスクの高さ。
        width: マスクの幅。
        num_blocks: モザイクブロックの数。
        min_block_size: モザイクブロックの最小サイズ。
        max_block_size: モザイクブロックの最大サイズ。

    Returns:
        モザイクマスクを表すNumPy配列。
    """
    mask = np.ones((height, width), dtype=np.uint8) * 255
    for _ in range(num_blocks):
        block_size = np.random.randint(min_block_size, max_block_size)
        x = np.random.randint(0, width - block_size)
        y = np.random.randint(0, height - block_size)
        cv2.rectangle(mask, (x, y), (x + block_size, y + block_size), 0, -1)
    return mask

def apply_mosaic_effect(frame, mask, block_size=30):
    """フレームにモザイク効果を適用する。

    Args:
        frame: モザイク効果を適用するフレーム。
        mask: モザイク効果を適用する領域を示すマスク。
        block_size: モザイクブロックのサイズ。

    Returns:
        モザイク効果が適用されたフレーム。
    """
    height, width = frame.shape[:2]
    small = cv2.resize(frame, (width // block_size, height // block_size))
    mosaic = cv2.resize(small, (width, height), interpolation=cv2.INTER_NEAREST)
    return np.where(mask[:,:,None] == 255, frame, mosaic)

def apply_blur_effect(frame, energy):
    """フレームにブラー効果を適用する。

    Args:
        frame: ブラー効果を適用するフレーム。
        energy: 音楽のエネルギーレベル。エネルギーレベルが高いほど、ブラーが強くなる。

    Returns:
        ブラー効果が適用されたフレーム。
    """
    blur_amount = int(energy * 4)
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

def apply_bounce_effect(frame, current_onset, max_onset):
    """フレームにバウンス効果を適用する。

    Args:
        frame: バウンス効果を適用するフレーム。
        current_onset: 現在のフレームのオンセット強度。
        max_onset: 音楽全体の最大オンセット強度。

    Returns:
        バウンス効果が適用されたフレーム。
    """
    bounce_amount = int(20 * current_onset / max_onset)
    if bounce_amount > 0:
        padded_frame = cv2.copyMakeBorder(frame, bounce_amount, bounce_amount, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        return padded_frame[bounce_amount:-bounce_amount, :]
    else:
        return frame

def apply_glitch_effect(frame, strength=10):
    """フレームにグリッチ効果を適用する。

    Args:
        frame: グリッチ効果を適用するフレーム。
        strength: グリッチの強度。

    Returns:
        グリッチ効果が適用されたフレーム。
    """
    height, width, _ = frame.shape
    glitch_frame = frame.copy()
    num_slices = np.random.randint(1, strength)
    for _ in range(num_slices):
        slice_height = np.random.randint(1, height // strength)
        start_y = np.random.randint(0, height - slice_height)
        start_x = np.random.randint(-strength, strength)
        end_x = width + start_x
        if start_x > 0:
            if end_x > width:
                end_x = width
            glitch_frame[start_y:start_y + slice_height, start_x:end_x] = frame[start_y:start_y + slice_height, :end_x - start_x]
        else:
            if -start_x > width:
                start_x = -width
            glitch_frame[start_y:start_y + slice_height, :end_x] = frame[start_y:start_y + slice_height, -start_x:]
    return glitch_frame

def process_frame(i, y, sr, S_dB, onset_env, img, particles, width, height, mosaic_mask, max_onset, prev_bars, zoom_factor):
    """単一のフレームを処理する。

    Args:
        i: 処理するフレームのインデックス。
        y: オーディオデータ。
        sr: サンプリングレート。
        S_dB: メルスペクトログラム。
        onset_env: オンセットエンベロープ。
        img: 背景画像。
        particles: パーティクルの情報を格納したNumPy配列。
        width: フレームの幅。
        height: フレームの高さ。
        mosaic_mask: モザイクマスク。
        max_onset: 音楽全体の最大オンセット強度。
        prev_bars: 前のフレームのバーの高さ。
        zoom_factor: ズーム係数。

    Returns:
        処理されたフレーム。
    """
    chunk_size = sr // 30
    chunk = y[i:i+chunk_size]
    spec_frame = S_dB[:, i//chunk_size] if i//chunk_size < S_dB.shape[1] else S_dB[:, -1]
    current_onset = onset_env[i//chunk_size] if i//chunk_size < len(onset_env) else onset_env[-1]
    energy = np.mean(np.abs(chunk)) * 10
    frame = img.copy()
    frame = apply_mosaic_effect(frame, mosaic_mask)
    particles = update_particles(particles, width, height, energy)
    draw_particles(frame, particles, (255, 255, 255))
    bar_width = width // 32 
    max_bar_height = height // 4

    # スペクトラムバーの描画
    for j, h in enumerate(spec_frame):
        target_height = int(np.interp(h, [S_dB.min(), S_dB.max()], [0, max_bar_height]))
        prev_bars[j] = prev_bars[j] * 0.7 + target_height * 0.3
        bar_height = int(prev_bars[j])
        
        bar_color = (255, 255, 255, 150)
        outline_color = (0, 0, 0, 255)

        overlay = frame.copy()
        cv2.rectangle(overlay, 
                      (j*bar_width, height // 2 - bar_height), 
                      ((j+1)*bar_width, height // 2), 
                      bar_color, 
                      -1)
        cv2.addWeighted(overlay, 0.5, frame, 1 - 0.5, 0, frame)
        cv2.rectangle(frame, 
                      (j*bar_width, height // 2 - bar_height), 
                      ((j+1)*bar_width, height // 2), 
                      outline_color, 
                      1)
        overlay = frame.copy()
        cv2.rectangle(overlay, 
                      (j*bar_width, height // 2), 
                      ((j+1)*bar_width, height // 2 + bar_height), 
                      bar_color, 
                      -1)
        cv2.addWeighted(overlay, 0.5, frame, 1 - 0.5, 0, frame)
        cv2.rectangle(frame, 
                      (j*bar_width, height // 2), 
                      ((j+1)*bar_width, height // 2 + bar_height), 
                      outline_color, 
                      1)

    overlay = frame.copy()
    cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)
    frame = apply_blur_effect(frame, energy)
    target_zoom = 1 + 0.05 * current_onset / max_onset
    zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3
    scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)
    start_y = (scaled_frame.shape[0] - height) // 2
    start_x = (scaled_frame.shape[1] - width) // 2
    frame = scaled_frame[start_y:start_y+height, start_x:start_x+width]
    frame = apply_bounce_effect(frame, current_onset, max_onset)
    if np.random.rand() < 0.1:
        frame = apply_glitch_effect(frame)

    return frame

def create_music_visualizer(image_path, audio_path, output_path):
    """音楽ビジュアライザーを作成する。

    Args:
        image_path: 背景画像のパス。
        audio_path: オーディオファイルのパス。
        output_path: 出力ビデオファイルのパス。
    """
    y, sr = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=32, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape
    particles = create_particles(100, width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('temp_output.mp4', fourcc, 30, (width, height))
    prev_bars = np.zeros(32)
    zoom_factor = 1.0
    mosaic_mask = create_mosaic_mask(height, width)
    max_onset = np.max(onset_env)

    chunk_size = sr // 30
    frame_args = [(i, y, sr, S_dB, onset_env, img, particles, width, height, mosaic_mask, max_onset, prev_bars, zoom_factor) 
                  for i in range(0, len(y), chunk_size)]

    # マルチスレッドでフレームを処理
    with ThreadPoolExecutor() as executor:
        frames = list(executor.map(lambda args: process_frame(*args), frame_args))

    for frame in frames:
        video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    video.release()
    video_clip = mpy.VideoFileClip("temp_output.mp4")
    audio_clip = mpy.AudioFileClip(audio_path)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    import os
    os.remove("temp_output.mp4")

# 使用例
image_path = 'image.png'
audio_path = 'audio.mp3'
output_path = 'output_video.mp4'
create_music_visualizer(image_path, audio_path, output_path)